# RNA 4 Nucleoside Prediction

In [3]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report

import numpy as np
from sklearn.metrics import classification_report, accuracy_score

from sklearn.metrics import mean_squared_error

from torch.utils.data import WeightedRandomSampler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import time
from collections import Counter
# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import pickle



In [2]:
! pip install optuna  scikit-learn gensim imbalanced-learn xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.0 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')
INPUT_TRAIN_IN = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/train_in.csv"
INPUT_TRAIN_OUT = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/train_out.csv"
INPUT_TEST_IN = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/test_in.csv"
INPUT_TEST_OUT = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/test_out.csv"
INPUT_VALIDATION_IN = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "/content/drive/My Drive/Colab Notebooks/Capstone Project/data/valid_out.csv"

# Record Constants
# INPUT_TRAIN_IN = "../../../data/train_in.csv"
# INPUT_TRAIN_OUT = "../../../data/train_out.csv"
# INPUT_TEST_IN = "../../../data/test_in.csv"
# INPUT_TEST_OUT = "../../../data/test_out.csv"
# INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
# INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'
ENCODING_FILE = '3-mer-dictionary.pkl'

WINDOW_SIZE = 50

# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence
# END - Modification is present at End of cropepd Sequence
CROP_STRATEGY = 'MID'

Mounted at /content/drive


In [6]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1  )
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1 )

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )


x_data = pd.concat([x_train_raw, x_test_raw, x_valid_raw], axis=0, ignore_index=True)
y_data = pd.concat([y_train_raw, y_test_raw, y_valid_raw], axis=0, ignore_index=True)

In [7]:
middle_index = (x_train_raw.shape[1] // 2) + 1 # This is location for Modified Sequence . Use this as Y Target

if CROP_STRATEGY == 'MID':
    STRAT_INEDX =middle_index - WINDOW_SIZE -1
    END_INDEX =middle_index + WINDOW_SIZE

if CROP_STRATEGY == 'END':
    STRAT_INEDX =middle_index - (WINDOW_SIZE*2) -1
    END_INDEX =middle_index # Ignore Modified Position

x_data_cropped =  x_data.iloc[:,STRAT_INEDX :END_INDEX]

concatenated_column= x_data_cropped.apply(lambda row: ''.join(map(str, row)), axis=1)
x_data_cropped = x_data_cropped.assign(Sequence=concatenated_column)

In [8]:
print(f" Shape of X : {x_data_cropped.shape} ")

 Shape of X : (309460, 102) 


### Encode with 3-mer Codon

In [9]:
kmer_dict = {}
k = 3
with open(ENCODING_FILE, 'rb') as f:
    kmer_dict = pickle.load(f)


def encode_with_k_mer_codon(sequence):
    encoded_sequence = []
    for i in range(len(sequence) - k + 1):
        encoded_sequence.append(kmer_dict[sequence[i:i+k]] )

    return np.array(encoded_sequence)

### Encode Target Class

In [10]:
RMs = ['A','G','C','T']
RMEncoding = [0,1,2,3]

All_RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
All_RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]


def convert_y_to_original_labels(row):
    label = ""
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            label = All_RMs[index]
    if label == '':
        return 'NonMoD'
    return label

def encode_target(y_data):
    # Write Customer Lable Encoder . This is required since we have train and test alreday splitted. Always creating a new instanc of label encoder will change encoding.
    y_encoded = []
    for y in y_data:
        index = RMs.index(y)
        encoding =  RMEncoding[index]
        y_encoded.append(encoding)
    return y_encoded

def prepare_data_for_binary_classification(x_data , y_data , prediction_class):
    # Convert One Hot Encoded Y to to Original Labels
    y_original_labels = y_data.apply(convert_y_to_original_labels,axis=1)
    x_data['Label'] = y_original_labels
    target_class = prediction_class
    selected_rna_data = x_data[x_data['Label'].isin(target_class)]

    y_filtered = selected_rna_data['Label']
    x_filtered = selected_rna_data.drop('Label', axis=1)

    return x_filtered , y_filtered


modification_list= ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol']

x_data_filtered , y_data_filtered = prepare_data_for_binary_classification(x_data_cropped , y_data , modification_list)


In [11]:
x_data_filtered.iloc[:,x_data_filtered.shape[1]-52]

0         A
1         A
2         A
3         A
4         A
         ..
309305    A
309306    A
309307    A
309308    A
309309    A
Name: 500, Length: 154853, dtype: object

In [13]:
x_data = None
y_data = None
y_valid_raw = None
x_valid_raw = None
x_train_raw = None
y_train_raw = None
x_test_raw = None
y_test_raw = None

In [14]:


print("Encode X")
X_encoded  =  x_data_filtered['Sequence'].apply(encode_with_k_mer_codon)
X_encoded = torch.tensor(X_encoded , dtype =  torch.long)


print("Encode Y")
y_originals = x_data_filtered.iloc[:,x_data_filtered.shape[1]-52] # Using Crop as MID Strategy - Get Middle Position as Target Label
print(y_originals.value_counts())
y_encoded = encode_target(y_originals)


print("Generate Train and Split..")
# Train set
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)

# Test and Validation set
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


Encode X


<ipython-input-14-e6286ae6e1e0>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X_encoded = torch.tensor(X_encoded , dtype =  torch.long)


Encode Y
A    138175
T      9086
C      5085
G      2507
Name: 500, dtype: int64
Generate Train and Split..


In [15]:
X_train.shape

torch.Size([108397, 99])

### Destroy Object for Memory Save

In [16]:
x_data_cropped = None
y_encoded = None
y_originals = None
X_encoded = None

### Apply Sampling on Data

In [17]:
y_train = torch.tensor(y_train , dtype=torch.long)
y_test = torch.tensor(y_test , dtype=torch.long)
y_valid = torch.tensor(y_valid , dtype=torch.long)

sm = RandomOverSampler(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
y_resampled = torch.tensor(y_resampled , dtype=torch.float32) # Keeping float32
X_resampled = torch.tensor(X_resampled , dtype=torch.float32)


In [18]:
X_resampled

tensor([[50., 50., 43.,  ...,  2., 33., 52.],
        [11., 60., 48.,  ..., 53., 59., 37.],
        [ 2., 61., 50.,  ..., 45., 51., 39.],
        ...,
        [53., 59., 37.,  ..., 48., 56., 55.],
        [50., 50., 50.,  ..., 26., 34., 35.],
        [ 1., 19., 56.,  ..., 56., 55., 61.]])

In [19]:
y_resampled_np = y_resampled.numpy()
# Get unique values and their counts
unique_values, counts = np.unique(y_resampled_np, return_counts=True)
# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

Value: 0.0, Count: 96680
Value: 1.0, Count: 96680
Value: 2.0, Count: 96680
Value: 3.0, Count: 96680


In [33]:
hyperparameter = {}
hyperparameter['INPUT_DIMENSION'] = len(kmer_dict) # For One Hot Encoding Input Dimension would be 4 as there only 4 unique nucleocide
hyperparameter['HIDDEN_DIMENSION'] = 128
hyperparameter['NO_OF_LAYERS'] = 1
hyperparameter['BATCH_SIZE'] = 32
hyperparameter['OUTPUT_DIMENSION'] = 4
hyperparameter['EMBEDDING_DIMENSION'] = 7 # if you are using Word2Vec Encoding then this should be same as Word2Vec Embedding Dim
hyperparameter['DROP_OUT'] = 0.1


In [31]:
class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

# Always use ReSampling for Training
train_dataset = RNADataset(X_resampled, y_resampled)
test_dataset = RNADataset(X_test, y_test)
valid_dataset = RNADataset(X_valid, y_valid)


train_dataloader = DataLoader(train_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

### RNA with Attention Mechanism.

In [22]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(BahdanauAttention, self).__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.unsqueeze(1)
        hidden = hidden.expand(-1, encoder_outputs.size(1), -1)
        alignment_scores = self.v(torch.tanh(self.W(hidden + encoder_outputs)))
        attention_weights = F.softmax(alignment_scores, dim=1)
        context_vector = torch.sum(attention_weights * encoder_outputs, dim=1)
        return context_vector, attention_weights

class RNABasicNucleosidePredictionModel(nn.Module):
    def __init__(self, input_dim,embedding_dim, hidden_dim, num_layers, output_dim, dropout=0.5):
        super(RNABasicNucleosidePredictionModel, self).__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.attention = BahdanauAttention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.long()
        x_embeded = self.embedding(x)

        lstm_out, (h, c) = self.lstm(x_embeded)

        hidden = h.squeeze(0)

        #apply attention using the last hidden state and the encoder outputs
        context_vector, attention_weights = self.attention(hidden, lstm_out)

        #dropout
        output = self.dropout(context_vector)

        #final output
        out = self.fc(output)

        return out


In [23]:
# class RNABasicNucleosidePredictionModel(nn.Module):
#     def __init__(self, input_dim,embedding_dim, hidden_dim,num_layers, output_dim , dropout=0.5):
#         super(RNABasicNucleosidePredictionModel, self).__init__()

#         # Pytorch Embedding
#         self.embedding = nn.Embedding(input_dim,embedding_dim)

#         # Bi-Directional LSTM Model
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout= dropout, batch_first=True )

#         # Fully Connected Layer
#         self.fc = nn.Linear(hidden_dim, output_dim)

#         # Drop out layer for Overfitting
#         self.dropout = nn.Dropout(dropout)


#     def forward(self, x):
#         x  = x.long()
#         x_embeded =  self.embedding(x)

#         lstm_out, (h,c) = self.lstm(x_embeded)

#         # Pass it to drop out layer
#         output = self.dropout(h[-1])

#         # Finally pass it to fully connected layer.
#         out = self.fc(output)

#         return out


def validate_model(model, test_dataloader , device ,loss_function):
    model.eval()
    running_loss = 0.0
    class_correct = [0] * hyperparameter['OUTPUT_DIMENSION']
    class_total = [0] * hyperparameter['OUTPUT_DIMENSION']
    total = 0
    correct = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.long().to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            true_labels.extend(labels.cpu().numpy())  # Capture True Lables for Summary Report
            predicted_labels.extend(predicted.cpu().numpy()) # Capture Predicted Labels Lables for Summary Report

    validation_loss = running_loss / len(test_dataloader)
    validation_accuracy = correct / total

    return validation_loss , validation_accuracy , true_labels , predicted_labels


def train_model(model, train_dataloader, test_dataloader, device, epochs, optimizer, loss_function):
    best_val_loss = float('inf')
    no_improvement_count = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        start_time = time.time()

        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.long().to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_dataloader)
        val_loss,  validation_accuracy , true_labels , predicted_labels = validate_model(model, test_dataloader, device, loss_function)
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Test Accuracy: {validation_accuracy:.4f} , Time Taken : {elapsed_time}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1
            if no_improvement_count == 10:
                print("No improvement in validation loss for 5 epochs. Training stopped.")
                break



In [34]:
model = RNABasicNucleosidePredictionModel(input_dim=hyperparameter['INPUT_DIMENSION'],
                            embedding_dim=hyperparameter['EMBEDDING_DIMENSION'],
                            hidden_dim=hyperparameter['HIDDEN_DIMENSION'] ,
                            num_layers = hyperparameter['NO_OF_LAYERS'],
                            output_dim=hyperparameter['OUTPUT_DIMENSION'],
                            dropout=hyperparameter['DROP_OUT'] )


loss_function = nn.CrossEntropyLoss()  ## MSELoss of Regression problem  # BCELoss for binary classification
optimizer = optim.Adam(model.parameters() ,  lr=0.0001)

# Number of Parameters for Model
total_parameters = []
for p in model.parameters():
    total_parameters.append(p.numel())

print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

# Train the model
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model Parameters  : " , hyperparameter)

# Train Model with configured Parameter
train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)

Total Number of Parameters for Model Training : 87959 
Model Parameters  :  {'INPUT_DIMENSION': 94, 'HIDDEN_DIMENSION': 128, 'NO_OF_LAYERS': 1, 'BATCH_SIZE': 32, 'OUTPUT_DIMENSION': 4, 'EMBEDDING_DIMENSION': 7, 'DROP_OUT': 0.1}
Epoch 1, Train Loss: 1.3099, Val Loss: 1.2770, Test Accuracy: 0.4526 , Time Taken : 45.182049036026
Epoch 2, Train Loss: 1.2819, Val Loss: 1.3413, Test Accuracy: 0.3318 , Time Taken : 45.96895098686218
Epoch 3, Train Loss: 0.9261, Val Loss: 0.4760, Test Accuracy: 0.8131 , Time Taken : 45.23568272590637
Epoch 4, Train Loss: 0.3026, Val Loss: 0.1128, Test Accuracy: 0.9718 , Time Taken : 45.2831084728241
Epoch 5, Train Loss: 0.1121, Val Loss: 0.0383, Test Accuracy: 0.9902 , Time Taken : 46.17015075683594
Epoch 6, Train Loss: 0.1275, Val Loss: 0.0281, Test Accuracy: 0.9935 , Time Taken : 45.27784252166748
Epoch 7, Train Loss: 0.0411, Val Loss: 0.0238, Test Accuracy: 0.9943 , Time Taken : 45.442524433135986
Epoch 8, Train Loss: 0.2531, Val Loss: 0.0261, Test Accuracy

In [35]:
# Evaluate the model on the test dataset
_, final_accuracy, true_labels, predicted_labels = validate_model(model, valid_dataloader,device,loss_function)

# Print the final accuracy
print(f"Final Accuracy: {final_accuracy:.4f}")

# Print the classification summary
print("\n Classification Summary:")
print(classification_report(true_labels, predicted_labels))

Final Accuracy: 1.0000

 Classification Summary:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20751
           1       1.00      1.00      1.00       359
           2       1.00      1.00      1.00       767
           3       1.00      1.00      1.00      1351

    accuracy                           1.00     23228
   macro avg       1.00      1.00      1.00     23228
weighted avg       1.00      1.00      1.00     23228



In [36]:
model_path =  '/content/drive/My Drive/Colab Notebooks/Capstone Project/model/4_nucleoside_model.pt'
torch.save(model, model_path)